# Multi Agent Collaboration - Search Orchestrator Agent

Multi-agent collaboration is a sophisticated AI architecture where multiple specialized agents work together in a coordinated manner to solve complex problems that exceed the capabilities of any single agent. In this distributed approach, each agent brings unique expertise and capabilities to the collective system, communicating and sharing information to achieve common goals.

In this workshop, we will create a Search Orchestrator agent that will act like a team leader, coordinating with three other specialized agents: 
1. FAQ Agent: For general questions about policies, procedures, or common inquiries about AnyCompany.
1. Product Search Agent: For queries related to finding specific products or product categories.
1. Inventory Agent: For questions about stock levels and availability.


> ℹ️ In this lab, we are going to run all the Agents in the same runtime. However, in a real life scenario you might run the Agents in different servers and have them communicate via [Streamable HTTP](https://strandsagents.com/latest/user-guide/concepts/tools/mcp-tools/#2-streamable-http).



![Orchestrator Agent](orchestrator.png)

We are going to reuse the collaborator agents that we created in the previous labs here. Their source code has been included in the current directory as python files.

The steps to complete this notebook are:

1. Install the necessary packages
1. Add environment variables
1. Create the collaborator Agents as @tools
1. Create the Search Orchestrator Agent
1. Test

## 1. Install the necessary packages

In [ ]:
!pip install --upgrade -q strands-agents strands-agents-tools boto3

## 2. Environment Configuration
The FAQ Agent and Product Search Agent use the [retrieve](https://github.com/strands-agents/tools/blob/main/src/strands_tools/retrieve.py) tool to fetch information from Amazon Bedrock Knowledge Bases. 

Find the respective Knowledge Base Ids from AWS Console and set in the below variables. Also set the AWS_REGION and MIN_SCORE environment variables needed for the ```retrieve``` tool.


In [ ]:
import os

# Fetch the knowledge base ID of the FAQ Knowledge Base
%store -r faq_kb_id
print(faq_kb_id)

# Fetch the knowledge base ID of the Product Search Knowledge Base from pre-requisites step
%store -r product_search_kb_id
print(product_search_kb_id)

# Also set the AWS REGION and MIN_SCORE needed for the retrieve tool.
region = "us-west-2"
os.environ["AWS_REGION"] = region #Change if needed
os.environ["MIN_SCORE"] = "0.4"

## 3. Create the Collaborator Agents

We will define 3 collaborator agents as local agents. Their source code, which we saw in the previous labs, have been included as python files in the same directory.

Import the strands libraries.

In [ ]:
from strands import Agent, tool
from strands_tools import agent_graph
from strands.models import BedrockModel
from botocore.config import Config as BotocoreConfig

#### Create the BedrockModel


In [ ]:
# Create a boto client config with custom settings
boto_config = BotocoreConfig(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)

# Create a Bedrock model instance
bedrock_model = BedrockModel(
    model_id="us.amazon.nova-premier-v1:0",
    region_name=region,  # try with different regions than the default - make sure you enable model access in the region you use
    temperature=0.3,
    top_p=0.8,
    boto_client_config=boto_config,
)

#### Declare the FAQ Agent as a Strands @tool
For this lab, we will be defining the Agents as `@tool`s. The code of `FAQ Agent`, `Product Inventory Agent` and `Product Search Agent` that we saw in the previous labs have been made available locally as 
1. `FAQAgent.py` 
1. `InventoryAgent.py` 
1. `ProductSearchAgent.py` 

respectively so you can focus on the orchestration part in this lab. 

All the Agents will be running in the same runtime. However, in a production scenario, each Agent could be running in a different server and communicating with each other using protocols like [A2A](https://a2aprotocol.ai/). 




> ℹ️ ***Tool Best Practices*** - 
> Language models rely heavily on tool descriptions to determine when and how to use them. Well-crafted descriptions significantly improve tool usage accuracy.
> A good tool description should:
> 1. Clearly explain the tool's purpose and functionality
> 1. Specify when the tool should be used
> 1. Detail the parameters it accepts and their formats
> 1. Describe the expected output format
> 1. Note any limitations or constraints

The FAQ Agent @tool simply passes the query to the actual Agent.


In [ ]:
# Import the FAQAgent from the local python in lab 4. This has the code that we saw in the FAQ Agents lab.
from FAQAgent import faq_agent

@tool
def faq_agent_tool(query: str) -> str:
    """
    Answers questions about AnyCompany.
    
    Use this tool when you need to find answers to general questions about the
    policies, procedures, or common inquiries of AnyCompany.

    Args:
        query: The search question. (String)
    
    Returns:
        The answer to the question (String)

    """
    print("Fetching from FAQ Knowledge Base")
    response = faq_agent(f"Use the knowledge base id {faq_kb_id}. {query}")
    print(f"The response from FAQ Agent is {response}")
    return response

#### Declare the Product Search and Inventory Agents


In [ ]:
# Import Product Search and Inventory Agents from local python files.
from ProductSearchAgent import product_search_agent
from InventoryAgent import inventory_agent


@tool
def product_search_agent_tool(query: str) -> str:
    """ 
    Discovers products for customers based on their requirements.

    Use this tool when you need to discover some products based
    on the requirements of the customer.

    This tool uses the Product Search Agent to find products.

    Example response:
    [
     { "product_id": "PROD-010", 
       "product_name": "Uniqlo Ultra Light Down Jacket", 
       "brand_name": "Uniqlo", 
       "category": "Clothing", 
       "subcategory": "Outerwear", 
       "gender": "Unisex", 
       "price": 69.90, 
       "sale_price": 49.90, 
       "size": ["XS", "S", "M", "L", "XL", "XXL"], 
       "color": ["Black", "Navy", "Red", "Olive", "Grey"], 
       "materials": ["100% Nylon", "Down filling", "Water-repellent coating"], 
       "season": "Fall/Winter" 
       },
       ...
    ]  

    Notes:
        - This tool only searches the products and does not provide
          inventory or availability information

    Args:
        query: The search requirements from the customer

    Returns:
        A list of matching product records, each containing:
        - id: Unique product identifier (string)
        - product_name: Product name
        - brand_name: Brand name
        - category: Category of the product
        - subcategory: Sub category
        - gender: Gender that the product applies to
        - price: Price
        - sale_price: Sales price
        - size: Available sizes as an array
        - color: Available colors as an array
        - materials: Materials
        - season: Season
    """
    print("Fetching from Product Search Knowledge Base using the query " + query)

    kbRes = product_search_agent(f"Use the knowledge base id {product_search_kb_id}. {query}")

    print(f"Products fetched from KB are : {kbRes}")
    return kbRes


@tool
def inventory_agent_tool(query: str) -> str:
    """ 
    Checks the inventory of products.

    Use this tool if you want to check if a product is in stock or not.

    This tool uses the Inventory Agent to find out if the product
    is in stock or not.

    Example response:
        {
            "product_id" : PRODUCT_ID,
            "in_stock": IN_STOCK_VALUE
        }
    
    Args:
        query: The product Id (String
    
    Returns:
        A JSON containing the fields:
        - product_id: Unique product identifier (string)
        - in_stock: "yes" or "no" indicating if the product is in stock or not.

    """
    print("Fetching from Inventory Agent with the query " + query)
    result = inventory_agent(query)
    print(f"Result from inventory_agent_tool is \" {result} \"")
    print("----------------------------")
    return result

## 4. Create the Search Orchestrator Agent

In [ ]:
# Search Orchestrator
ORCHESTRATOR_SYSTEM_PROMPT = """You are the Search Orchestrator Agent, a sophisticated AI coordinator 
responsible for managing and directing queries to three specialized collaborator agents: the FAQ Agent, 
the Product Search Agent, and the Inventory Agent. Your primary role is to efficiently handle user 
inquiries by determining which agent(s) to engage and in what order.

Your key responsibilities include:
1. Analyze incoming user queries to understand their intent and requirements.
2. Decide which tool(s) to consult based on the nature of the query:
   - FAQ Agent Tool: For general questions about policies, procedures, or common inquiries. This tool does 
                     not have product or product availability related information.
   - Product Search Agent Tool: For queries related to finding specific products or product categories.
   - Inventory Agent Tool: For questions about stock levels and availability.
3. Determine the optimal sequence for consulting these tools if multiple tools are needed.
4. Formulate clear, concise sub-queries for each relevant tool.
5. Collect and synthesize responses from the collaborator tools.
6. Generate a comprehensive, coherent response for the user based on the information gathered.

Note:
1. Fetch product availability information only when explicitly asked by the customer. 
2. FAQ Agent Tool does not have product or product availability related information.

Remember, your goal is to provide accurate, helpful, and efficient responses to user queries by 
leveraging the specialized knowledge of each collaborator tool. Always strive for clarity, 
conciseness, and relevance in your communications with both users and collaborator tools.
"""

# Create the orchestrator agent with the collaborator Agents as tools.
orchestrator = Agent(
    system_prompt=ORCHESTRATOR_SYSTEM_PROMPT,
    tools=[faq_agent_tool, product_search_agent_tool, inventory_agent_tool],
    model=bedrock_model
)

In [ ]:
result1 = orchestrator("What is the returns policy of AnyCompany?")

In [ ]:
result2 = orchestrator("Product recommendations for clothes")

In [ ]:
result3 = orchestrator("Hello, I am going to attend a cocktail party, and would like some suggestions for smart casual jacket. What can you recommend me? Also check if that's in inventory.")

# End of Workshop
Congrats!! You have successfully completed all the labs.